In [1]:
import pandas as pd
import time
import bs4
import random
import requests


In [1]:
!pip install selenium

In [2]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

In [3]:
driver = webdriver.Firefox()

In [216]:
token = 'https://www.seloger.com/immobilier/achat/immo-bordeaux-33/bien-maison/?LISTING-LISTpg='

In [217]:
def get_pages(token, nb):
    pages = []
    for i in range(1,nb+1):
        j = token + str(i)
        pages.append(j)
    return pages

In [218]:
nb_pages = 1

In [219]:
pages = get_pages(token,nb_pages)

In [220]:
pages

['https://www.seloger.com/immobilier/achat/immo-bordeaux-33/bien-maison/?LISTING-LISTpg=1']

In [221]:
pages[0]

'https://www.seloger.com/immobilier/achat/immo-bordeaux-33/bien-maison/?LISTING-LISTpg=1'

In [222]:
driver.get(pages[0])

In [223]:
src = driver.page_source

In [224]:
soup = bs4.BeautifulSoup(src, 'html.parser')
soup

<html class="windows desktop landscape" lang="fr"><head>
<meta charset="utf-8"/>
<meta content="width=device-width, initial-scale=1.0, user-scalable=no" name="viewport"/>
<meta content="Toutes les annonces d'achat de maison à Bordeaux (33). Filtrez votre recherche de maison à Bordeaux (33) par nombre de pièces, chambres, superficie, avec balcon, cave, au dernier étage, avec gardien ou jardin, ..." name="description"/>
<title>Vente maisons Bordeaux (33) | Acheter maisons à Bordeaux 33000</title>
<link href="https://cdnjs.cloudflare.com" rel="dns-prefetch"/>
<link href="https://static-seloger.com" rel="dns-prefetch"/>
<link href="https://www.seloger.com/favicon.ico" rel="shortcut icon"/>
<link href="/list/css/font-icon.css" rel="stylesheet" type="text/css"/>
<link href="/commoncomp/headersl/css/2.10.2-main.css" rel="stylesheet" type="text/css"/>
<link href="/list/css/framework.min.css" rel="stylesheet" type="text/css"/>
<style data-styled="fVWLgO epIjkB joMHSe igZifn NzOwZ LLczN bsiXpA j

In [225]:
em_box2 = soup.find_all("span")
em_box2[240-55]

<span>Saint Bruno-Saint Augustin</span>

In [227]:
nb_annonces = em_box2[240-67].text.split("a")[0].strip(" ")
nb_annonces = int(nb_annonces[0]+nb_annonces[2:5])
nb_annonces

1166

In [274]:
nb_pages = round(nb_annonces / 25)
nb_pages

47

In [276]:
pages = get_pages(token,nb_pages)
len(pages)

47

In [285]:
from tqdm import tqdm

annonces_list_url =[]

for page in tqdm(range(nb_pages)) :
    driver.get(pages[page])
    src = driver.page_source
    soup = bs4.BeautifulSoup(src, 'html.parser')
    url_search = soup.find_all(class_="swiper-container")
    for url in range(len(url_search)) :
        if "www.seloger.com" in str(url_search[url].contents[1]) :
            annonces_list_url.append(str(url_search[url].contents[1]).split("href=")[1].split('"')[1])
    
    
#    print(f"page {page+1} proceed")


100%|██████████| 47/47 [02:24<00:00,  3.08s/it]


In [292]:
len(url_search)

16

In [286]:
len(annonces_list_url)

100

In [293]:
annonces_list_url

['https://www.seloger.com/annonces/achat/maison/bordeaux-33/saint-bruno-saint-augustin/149067859.htm',
 'https://www.seloger.com/annonces/achat/maison/bordeaux-33/cauderan-barriere-judaique/155293763.htm',
 'https://www.seloger.com/annonces/achat/maison/bordeaux-33/cauderan-barriere-judaique/154033643.htm',
 'https://www.seloger.com/annonces/achat/maison/bordeaux-33/la-bastide/153480241.htm',
 'https://www.seloger.com/annonces/achat-de-prestige/maison/bordeaux-33/155123823.htm',
 'https://www.seloger.com/annonces/achat/maison/bordeaux-33/155498961.htm',
 'https://www.seloger.com/annonces/achat/maison/bordeaux-33/153545643.htm',
 'https://www.seloger.com/annonces/achat-de-prestige/maison/bordeaux-33/saint-bruno-saint-augustin/148761345.htm',
 'https://www.seloger.com/annonces/achat/maison/bordeaux-33/saint-bruno-saint-augustin/150760229.htm',
 'https://www.seloger.com/annonces/achat-de-prestige/maison/bordeaux-33/saint-bruno-saint-augustin/141835655.htm',
 'https://www.seloger.com/annon

In [297]:
import json
annonces_attributs = []

for annonce_url in tqdm(range(len(annonces_list_url))):
    driver.get(annonces_list_url[annonce_url])
    src_annonce = driver.page_source
    soup_annonce = bs4.BeautifulSoup(src_annonce, 'html.parser')
    scripts_annonce = soup_annonce.find_all("script")
    for script in scripts_annonce:
        if len(script.contents) > 0 :
            lines = script.contents[0].splitlines()
            if len(lines) > 1:
                if "var ava_data" in lines[1]:
                    data = script.contents[0]
                    data = data[data.find("products : ["):data.find("//")].strip("products : [")
                    json_data = json.loads(data)
    annonces_attributs.append(json_data)

100%|██████████| 100/100 [03:24<00:00,  2.05s/it]


In [298]:
len(annonces_attributs)

100

In [299]:
annonces_attributs[0]

{'idannonce': '149067859',
 'idagence': '43414',
 'idtiers': '25863',
 'typedebien': 'Maison / Villa',
 'typedetransaction': ['vente'],
 'idtypepublicationsourcecouplage': 'SL',
 'position': '0',
 'codepostal': '33000',
 'produitsvisibilite': 'AD:AC:BB:AD:AC:BB:AW',
 'affichagetype': [{'name': 'detail', 'value': 'true'}],
 'cp': '33000',
 'codeinsee': '330063',
 'ville': 'bordeaux',
 'etage': '0',
 'idtypechauffage': 'gaz',
 'idtypecommerce': '0',
 'idtypecuisine': 'équipée',
 'naturebien': '1',
 'si_balcon': '0',
 'nb_chambres': '3',
 'nb_pieces': '4',
 'si_sdbain': '1',
 'si_sdEau': '0',
 'nb_photos': '6',
 'prix': '444000',
 'mensualiteFinance': '1\xa0839',
 'surface': '91',
 'dpeL': 'E',
 'dpeC': '252'}

In [315]:
df = pd.DataFrame(annonces_attributs)
df.pop("idtiers")
df.pop("idagence")
df.pop("idtypepublicationsourcecouplage")
df.pop("position")
df.pop("produitsvisibilite")
df.pop("affichagetype")
df.pop("cp")
df.pop("codeinsee")
df.pop("idtypecommerce")
df.pop("mensualiteFinance")
df

,idannonce,typedebien,typedetransaction,codepostal,ville,etage,idtypechauffage,idtypecuisine,naturebien,si_balcon,nb_chambres,nb_pieces,si_sdbain,si_sdEau,nb_photos,prix,surface,dpeL,dpeC
0,149067859,Maison / Villa,[vente],33000,bordeaux,0,gaz,équipée,1,0,3,4,1,0,6,444000,91,E,252
1,155293763,Maison / Villa,[vente],33200,bordeaux,0,gaz,équipée,1,0,2,4,1,1,8,432000,76,D,169
2,154033643,Maison / Villa,[vente],33200,bordeaux,0,gaz,0,1,0,3,4,1,1,6,514000,110,0,0
3,153480241,Maison / Villa,[vente],33000,bordeaux,0,individuel gaz,américaine,1,0,3,4,1,0,4,550000,110,0,0
4,155123823,Maison / Villa,[vente de prestige],33200,bordeaux,0,0,américaine,1,0,3,4,1,1,4,780000,120,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,152549445,Maison / Villa,[vente],33800,bordeaux,0,0,aucune,1,0,3,4,1,1,1,420000,88,0,0
96,150125363,Maison / Villa,[vente],33000,bordeaux,0,0,séparée,1,0,2,3,1,1,2,525000,78,0,0
97,154198679,Maison / Villa,[vente de prestige],33300,bordeaux,0,individuel gaz radiateur,aucune,1,0,3,5,0,0,1,630000,125,0,0
98,154873431,Maison / Villa,[vente],33000,bordeaux,0,0,américaine,1,0,3,4,1,1,1,378000,92,0,0


In [323]:
file = df.to_csv("annonces_features.csv", sep=";")

In [14]:
quartier = soup_page.find(class_ = "detail-title title1").text.split("²")[1].split("\n")[0]
quartier

' Bordeaux - Le Lac-Bacalan'